In [53]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

COLUMNA_GRD_INTERNO = [
    "rut",
    "sexo_(cod)",
    "prevision_(cod)",
    "hospital_de_procedencia_(cod)",
    "fecha_ingreso",
    "fecha_de_egreso",
    "ano_de_egreso",
    "estancias_totales",
    "diag_01_principal_(cod)",
    "diag_01_principal_(des)",
    "diag_02_secundario_(cod)",
    "diag_02_secundario_(des)",
    "diag_03_secundario_(cod)",
    "diag_03_secundario_(des)",
    "proced_01_principal_(cod)",
    "proced_01_principal_(des)",
]

In [55]:
# Lee GRD del INT
df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_sabanas["FECHA_INGRESO"] = pd.to_datetime(df_sabanas["FECHA_INGRESO"])
df_sabanas["FECHA_EGRESO"] = pd.to_datetime(df_sabanas["FECHA_EGRESO"])

# Lee GRD Interno del INT
df_grd_interno = pd.read_csv("../data/processed/grd_interno.csv", usecols=COLUMNA_GRD_INTERNO)
# Formatea fechas de GRD
df_grd_interno["fecha_ingreso"] = pd.to_datetime(df_grd_interno["fecha_ingreso"])
df_grd_interno["fecha_de_egreso"] = pd.to_datetime(df_grd_interno["fecha_de_egreso"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_87913/2510431287.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sabanas = pd.read_csv("../data/processed/grd_sabanas.csv", usecols=COLUMNAS_GRD)


In [56]:
# Limpia la base de pabellon, sacando las suspendidas y sin duracion
df_pabellon_limpio = df_pabellon.copy()
df_pabellon_limpio = df_pabellon_limpio.query("suspendida == 'NO' and duracion.notna()")

In [57]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_sabanas.merge(
    df_pabellon_limpio, how="inner", left_on="RUT", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [58]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe()
    .reset_index()
)

In [63]:
COLUMNAS_A_MOSTRAR = [
        # "rut",
        "fecha_ingreso",
        "fecha_de_egreso",
        "ano_de_egreso",
        "estancias_totales",
        "diag_01_principal_(des)",
        "proced_01_principal_(cod)",
        "proced_01_principal_(des)",
        "fecha",
        "primer_diagnostico",
        "nombre_de_la_operacion",
        "duracion"
    ]

# Une la base de GRD Interno y Pabellon segun el RUT del paciente
grd_interno_y_pabellon = df_grd_interno.merge(
    df_pabellon_limpio, how="inner", left_on="rut", right_on="ID_PACIENTE"
)

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_interno_y_pabellon = grd_interno_y_pabellon.query(
    "fecha >= fecha_ingreso and fecha <= fecha_de_egreso"
)

In [64]:
grd_interno_y_pabellon[COLUMNAS_A_MOSTRAR]

,fecha_ingreso,fecha_de_egreso,ano_de_egreso,estancias_totales,diag_01_principal_(des),proced_01_principal_(cod),proced_01_principal_(des),fecha,primer_diagnostico,nombre_de_la_operacion,duracion
0,2015-08-28,2015-09-15,2015,18,Derrame pleural no clasificado en otra parte,34.21,TORACOSCOPIA TRANSPLEURAL,2015-09-09,NEOPLASIA TIROIDEA AVANZADA,VTC - BP,0 days 01:15:00
3,2015-09-09,2015-09-15,2015,6,Neuropatía intercostal,4.81,INYECCION ANESTESICO NERVIO PERIFERICO PARA AN...,2015-09-14,NEURITIS INTERCOSTAL,INFILTRACION INTERCOSTAL,0 days 00:26:00
11,2015-09-07,2015-09-15,2015,8,Tumor maligno secundario del pulmón,32.29,ESCISION PULMON LOCAL.OTRA,2015-09-11,MTTS LSI,METASTASECTOMIA,0 days 02:50:00
12,2015-08-19,2015-09-14,2015,26,Bocio multinodular no tóxico,6.40,TIROIDECTOMIA TOTAL,2015-09-09,TU DE MEDIASTINO ANTERIOR,CERVICO ESTERNOTOMIA - TIMECTOMIA TOTAL,0 days 07:50:00
13,2015-09-09,2015-09-16,2015,7,Estenosis mitral con insuficiencia,35.23,SUSTITUCION VALVULA MITRAL CON INJERTO,2015-09-10,CANAL AV PARCIAL,ESTERNOTOMIA X 3 - RECAMBIO VALV MITRAL BIOLOGICA,0 days 06:05:00
...,...,...,...,...,...,...,...,...,...,...,...
33068,2022-08-26,2022-09-06,2022,11,Disección de aorta (cualquier parte),39.71,REPARACION ENDOVASCULAR AORTA ABDOMINAL CON IN...,2022-09-02,ANEURISMA ABDOMINAL,ASEO QUIRURGICO,0 days 01:40:00
33070,2022-08-31,2022-09-05,2022,5,"Tumor maligno del lóbulo superior, bronquio o ...",33.20,BIOPSIA PULMON TORACOSCOPICA,2022-09-01,NODULO LOBULO SUP DER,BP PULMONAR LOBULO SUP DER,0 days 01:45:00
33072,2022-08-30,2022-09-04,2022,5,"Tumor maligno del lóbulo inferior, bronquio o ...",32.39,RESECCION PULMON SEGMENTARIA.OTRA Y NEOM,2022-08-31,METASTASIS PULMONAR LII,RESECCION METASTASIS X TCT,0 days 04:00:00
33074,2022-08-23,2022-09-07,2022,15,Cardiomiopatía hipertrófica obstructiva,37.34,ESCISION LESION CORAZON ABORDAJE ENDOVASCULAR ...,2022-08-31,MIOCARDIOPATIA HIPERTROFICA OBSTRUCTIVA,MIECTOMIA SEPTAL - ALFIERE MITRAL,0 days 04:40:00
